## Подготовка

Подключим необходимые библиотеки.

In [22]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from scipy.stats import loguniform
from scipy.stats import uniform
from lightgbm import LGBMClassifier

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rus95\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Загрузка и ознакомление с данными

Загрузим набор данных и посмотрим на первые 5 строк.

In [2]:
df = pd.read_csv('toxic_comments.csv')

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Посмотрим на распределение целевого признака.

In [5]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

- В наборе данных 159571 строка, пропусков нет.
- Имеем дело с английскими текстами.
- Имеется дисбаланс классов. Токсичных комментариев всего 10%.

### Лемматизация

Перед лемматизацией удалим из текстов все ненужные символы, оставим лишь буквы латинского алфавита и пробелы.

In [6]:
corpus = df['text']

In [7]:
corpus = corpus.apply(lambda x: ' '.join(re.sub(r'[^a-zA-Z ]', ' ', x).split()))

Заменим каждый текст строкой, состоящей из лемм его слов.

In [8]:
lemmatizer = WordNetLemmatizer()

corpus = corpus.apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

Приведем все тексты к нижнему регистру.

In [9]:
corpus = corpus.str.lower()

Посмотрим на первые 5 строк до и после преобразований.

In [12]:
pd.concat([df['text'], corpus], axis=1).head()

,text,text
0,Explanation\nWhy the edits made under my usern...,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",you sir are my hero any chance you remember wh...


### Разбиение на обучающую и тестовую выборки

Разобьем данные на обучающую и тестовую выборки в соотношении 80:20.

In [13]:
features_train, features_test, target_train, target_test = train_test_split(corpus, 
                                                                            df['toxic'], 
                                                                            test_size=0.2, 
                                                                            random_state=42)

Проверим размеры.

In [14]:
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(127656,)
(31915,)
(127656,)
(31915,)


Также проверим долю "токсичных" комментариев в обеих выборках.

In [15]:
print(target_train.mean())
print(target_test.mean())

0.10168734724572288
0.10164499451668495


Разбиение произведено корректно.

Установим стоп-слова для английского языка для дальнейшей векторизации текстов.

In [16]:
stop_words = set(stopwords.words('english'))

**Вывод:**
1. Данные были загружены. Имеем 143916 строк с английским текстом, и значение целевого признака (токсичен комментарий или нет) для каждого текста.
2. Из текстов были удалены лишние символы, а также проведена лемматизация.
3. Данные были разбиты на обучающую и тестовую выборки в соотношении 80:20.

## Обучение

Обучим несколько моделей, а также оптимизируем их гиперпараметры с помощью техники Randomized Search. 

Интересующая нас метрика - F1-мера. 

### Логистическая регрессия

Начнем с логистической регрессии.

Чтобы кросс-валидация была выполнена корректно, создадим пайплайн, состоящий из двух шагов: векторизация и сама модель. Это нужно, чтобы при каждом разбиении валидации векторизация производилась лишь по обучающей выборке.

Векторизацию выполним с помощью техники TF-IDF, будем использовать лишь униграммы.

In [24]:
pipe_log_reg = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 1))), 
                         ('log_reg', LogisticRegression(class_weight='balanced', max_iter=1000))])

In [25]:
params_log_reg = {
    'log_reg__C' : loguniform(0.01, 100)
}

In [27]:
rand_search_lr = RandomizedSearchCV(
    estimator=pipe_log_reg,
    param_distributions=params_log_reg,
    n_iter=20,
    scoring='f1',
    n_jobs=-1,
    cv=5,
    verbose=1,
    random_state=42
)

rand_search_lr.fit(features_train, target_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Wall time: 3min 9s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vectorizer',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                

Наилучший параметр.

In [28]:
rand_search_lr.best_params_

{'log_reg__C': 6.796578090758152}

Наилучшее значение F1-меры на кросс-валидации.

In [29]:
f1_log_reg_cv = rand_search_lr.best_score_
f1_log_reg_cv

0.7598871352929937

Сохраним наилучшую модель.

In [30]:
log_reg = rand_search_lr.best_estimator_

### Случайный лес

Как и в предыдущем случае, создаем пайплайн.

In [34]:
pipe_rf = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 1))), 
                    ('random_forest', RandomForestClassifier(class_weight='balanced', random_state=42))])

В случайном лесу будем калибровать глубину дерева, их количество, а также ограничения на сплиты и листы.

In [35]:
params_rand_forest = {
    'random_forest__max_depth' : range(6, 18),
    'random_forest__n_estimators' : range(5, 200),
    'random_forest__min_samples_split' : range(2, 8),
    'random_forest__min_samples_leaf' : range(1, 6)
}

In [36]:
rand_search_rf = RandomizedSearchCV(
    estimator=pipe_rf,
    param_distributions=params_rand_forest,
    n_iter=20,
    scoring='f1',
    n_jobs=-1,
    cv=5,
    verbose=1,
    random_state=42
)

rand_search_rf.fit(features_train, target_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Wall time: 2min 4s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vectorizer',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                

Наилучшие параметры модели.

In [37]:
rand_search_rf.best_params_

{'random_forest__n_estimators': 190,
 'random_forest__min_samples_split': 4,
 'random_forest__min_samples_leaf': 1,
 'random_forest__max_depth': 17}

Наилучшая производительность модели.

In [38]:
f1_random_forest_cv = rand_search_rf.best_score_
f1_random_forest_cv

0.38906837488718793

Сохраним самую успешную модель.

In [39]:
random_forest = rand_search_rf.best_estimator_

### Градиентный бустинг

In [40]:
pipe_lgbm = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 1))), 
                      ('lgbm', LGBMClassifier(class_weight='balanced', random_state=42))])

В случае градиентного бустинга будем калибровать глубину деревьев, их количество, скорость обучения и коэффициент L2-регуляризации. Будем использовать библиотеку LightGBM.

In [41]:
params_lgbm = {
    'lgbm__max_depth' : range(3, 6),
    'lgbm__n_estimators' : range(5, 350),
    'lgbm__learning_rate' : loguniform(0.0001, 0.5),
    'lgbm__reg_lambda' : uniform(0.01, 10)
}

In [42]:
rand_search_lgbm = RandomizedSearchCV(
    estimator=pipe_lgbm,
    param_distributions=params_lgbm,
    n_iter=50,
    scoring='f1',
    n_jobs=-1,
    cv=5,
    verbose=1,
    random_state=42
)

rand_search_lgbm.fit(features_train, target_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Wall time: 20min 53s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vectorizer',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                

Наилучшие параметры.

In [43]:
rand_search_lgbm.best_params_

{'lgbm__learning_rate': 0.409703267332127,
 'lgbm__max_depth': 4,
 'lgbm__n_estimators': 347,
 'lgbm__reg_lambda': 2.7964646423661144}

Наилучшее значение F1-меры.

In [44]:
f1_lgbm_cv = rand_search_lgbm.best_score_
f1_lgbm_cv

0.7413839578965877

Сохраним наилучшую модель.

In [45]:
lgbm = rand_search_lgbm.best_estimator_

### Сравнение моделй

Сведем полученные на кросс-валидации результаты в одну таблицу.

In [46]:
results = pd.DataFrame([
    ['logistic regression', f1_log_reg_cv],
    ['random forest', f1_random_forest_cv],
    ['LightGBM', f1_lgbm_cv]
], columns=['model', 'f1 on cross-validation']
)
results

,model,f1 on cross-validation
0,logistic regression,0.759887
1,random forest,0.389068
2,LightGBM,0.741384


- Случайный лес показал самую худшую производительность на кросс-валидации.
- Лучше всех себя проявила логистическая регрессия. Градиентный бустинг показал сравнимые результаты.

### Тестирование моделей

Сделаем предсказания на тестовой выборке.

In [48]:
y_pred_lr = log_reg.predict(features_test)
y_pred_rf = random_forest.predict(features_test)
y_pred_lgbm = lgbm.predict(features_test)

Посчитаем F1-меру.

In [50]:
f1_log_reg_test = f1_score(target_test, y_pred_lr)
f1_random_forest_test = f1_score(target_test, y_pred_rf)
f1_lgbm_test = f1_score(target_test, y_pred_lgbm)

Добавим полученные значения в таблицу.

In [51]:
results['f1 on test'] = [f1_log_reg_test, f1_random_forest_test, f1_lgbm_test]

In [52]:
results

,model,f1 on cross-validation,f1 on test
0,logistic regression,0.759887,0.765383
1,random forest,0.389068,0.384524
2,LightGBM,0.741384,0.747125


Такая же ситуация, как и на кросс-валидации.

Также посчитаем точность, полноту и accuracy.

In [53]:
results['recall on test'] = [recall_score(target_test, y_pred_lr), 
                             recall_score(target_test, y_pred_rf), 
                             recall_score(target_test, y_pred_lgbm)]

results['precision on test'] = [precision_score(target_test, y_pred_lr), 
                                precision_score(target_test, y_pred_rf), 
                                precision_score(target_test, y_pred_lgbm)]

results['accuracy on test'] = [accuracy_score(target_test, y_pred_lr), 
                               accuracy_score(target_test, y_pred_rf), 
                               accuracy_score(target_test, y_pred_lgbm)]

In [54]:
results

,model,f1 on cross-validation,f1 on test,recall on test,precision on test,accuracy on test
0,logistic regression,0.759887,0.765383,0.830148,0.709992,0.948269
1,random forest,0.389068,0.384524,0.845561,0.248843,0.724863
2,LightGBM,0.741384,0.747125,0.831073,0.678580,0.942817


Все три модели имеют сравнимую довольно высокую полноту, однако случайный лес имеет слишком низкую точность, что говорит о том, что модель слишком часто предсказывает класс "1", когда она этого делать не должна, т.е. высока доля ложно-положительных ответов.

По условию задачи, нужно получить F1-меру выше 0.75, что и было достигнуто с помощью логистической регрессии.

## Выводы

- Исходные текстовые данные были очищены от ненужных символов, затем лемматизированы и переведены в векторы с помощью техники TF-IDF.
- Три модели машинного обучения были обучены: логистическая регрессия, случайный лес и градиентный бустинг. Были оптимизированы их гиперпараметры.
- Наивысшую F1-меру на кросс-валидации и тесте показала логистическая регрессия, ее значение превышает заданные заказчиком 0.75, так что можем считать нашу цель достигнутой.
- Градиентный бустинг показал неплохие результаты, но не смог достичь отметки 0.75. Возможно, этого бы удалось добиться, если бы мы использовали больше итераций на этапе поиска оптимальных гиперпараметров, однако из-за болього размера признакого пространства это бы заняло много времени.
- Случайный лес показал наихудшие результаты. Дело в том, что модель слишком часто предсказывает класс "1", из-за чего высока доля ложно-положительных ответов.